In [75]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

# authorization
cid = '5b85a9af539f47da8dcbbcf517e42650'
secret = 'cb3ce73abd464866b285015d631ca1d5'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [76]:
def call_playlist(playlist_url, limit, offset):
    # NOTE: use playlist_url instead of playlist_id. playlist_id doesn't accept limit and offset parameters for some reason?
    
    # create an empty list and an empty df
    playlist_features_list = ['artist', 'album', 'track_name', 'track_id', 'danceability', 'energy', 'loudness',
                              'valence', 'tempo', 'duration_ms', 'acousticness']
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    # loop through the specified playlist and extract wanted features
    playlist = sp.user_playlist_tracks('', playlist_url, limit=limit, offset=offset)["items"]
    for track in playlist:

        # create empty dict
        playlist_features = {}

        # get metadata
        playlist_features['artist'] = track['track']['album']['artists'][0]['name']
        playlist_features['album'] = track['track']['album']['name']
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
#        playlist_features["explicit"] = track["track"]["explicit"]
        playlist_features["popularity"] = track["track"]["popularity"]
        playlist_features["album_release_date"] = track["track"]["album"]["release_date"]
#         playlist_features["duration_ms"] = track["track"]["duration_ms"]
#         playlist_features['added_by'] = track["added_by"]["id"]
#         playlist_features['added_at'] = track["added_at"]
        
        # get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[4:]:
            playlist_features[feature] = audio_features[feature]
        
        # concat dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)

    # return df
    return playlist_df

In [77]:
first100 = call_playlist('https://open.spotify.com/playlist/2U9q9cml8EInaiYFDYwZ73?si=1f188bb952c041c1', 100, 0)

In [78]:
hungryGuysRadio.head(13)

,artist,album,track_name,track_id,danceability,energy,loudness,valence,tempo,duration_ms,acousticness,popularity,album_release_date
0,Various Artists,Happy New Year,India Waale,69EoR3Z8jgRODL3AmQs5ZP,0.720,0.75300,-5.453,0.6190,128.016,237715,0.037800,46.0,2014-09-26
1,The Kid LAROI,F*CK LOVE 3: OVER YOU,STAY (with Justin Bieber),5PjdY0CKGZdEuoNab3yDmX,0.591,0.76400,-5.484,0.4780,169.928,141806,0.038300,93.0,2021-07-23
2,THE ANXIETY,THE ANXIETY,Meet Me At Our Spot,07MDkzWARZaLEdKxo6yArG,0.773,0.47000,-7.930,0.3990,94.995,162680,0.015300,92.0,2020-03-13
3,The Weeknd,After Hours,Blinding Lights,0VjIjW4GlUZAMYd2vXMi3b,0.514,0.73000,-5.934,0.3340,171.005,200040,0.001460,92.0,2020-03-20
4,Dua Lipa,Future Nostalgia,Levitating (feat. DaBaby),5nujrmhLynf4yMoMtj8AQF,0.702,0.82500,-3.787,0.9150,102.977,203064,0.008830,88.0,2020-03-27
5,Drake,Certified Lover Boy,Way 2 Sexy (with Future & Young Thug),0k1WUmIRnG3xU6fvvDVfRG,0.803,0.59700,-6.035,0.3310,136.008,257605,0.000619,93.0,2021-09-03
6,Playboi Carti,Whole Lotta Red,Sky,29TPjc8wxfz4XMn21O7VsZ,0.785,0.90300,-4.184,0.5650,139.980,193234,0.258000,85.0,2020-12-25
7,Ludwig van Beethoven,"Beethoven: Piano Sonatas, Vol.3","Sonata No. 14 ""Moonlight"" in C-Sharp Minor"", O...",3DNRdudZ2SstnDCVKFdXxG,0.184,0.00527,-37.264,0.1510,170.612,315427,0.995000,69.0,2007-09-25
8,Mariah Carey,Merry Christmas,All I Want for Christmas Is You,0bYg9bo50gSsH3LtXe2SQn,0.336,0.62700,-7.463,0.3500,150.273,241107,0.164000,76.0,1994-11-01
9,Céline Dion,Let's Talk About Love,"My Heart Will Go On - Love Theme from ""Titanic""",33LC84JgLvK2KuW43MfaNq,0.428,0.27600,-11.729,0.0382,99.195,280000,0.732000,75.0,1997-10-24
